# Initialize packages

In [1]:
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:
def parse_filband(feig, npl=10):
    # feig : filband in bands.x input file
    # npl : number per line, 10 for bands.x, 6 for phonon

    f=open(feig,'r')
    lines = f.readlines()

    header = lines[0].strip()
    line = header.strip('\n')
    shape = re.split('[,=/]', line)
    nbnd = int(shape[1])
    nks = int(shape[3])
    eig = np.zeros((nks, nbnd+1), dtype=np.float32)

    dividend = nbnd
    divisor = npl
    div = nbnd // npl + 1 if nbnd % npl == 0 else nbnd // npl + 2 
    kinfo=[]
    for index, value in enumerate(lines[1:]):
        value = value.strip(' \n')
        quotient = index // div
        remainder = index % div

        if remainder == 0:
            kinfo.append(value)
        else:
            value = re.split('[ ]+', value)
            a = (remainder - 1) * npl
            b = a + len(value)
            eig[quotient][a:b] = value

    f.close()

    return eig, nbnd, nks, kinfo

e_ref= 1.6406 # set to fermi-level in scf output for metal, only applicable for do_find_gap=False

band_plot = plt.figure(figsize=(7,5))
ymin=-6  # y range in plot
ymax=4
lw=1 # line width

eig, nbnd, nks, kinfo = parse_filband('mos2.bands.dat')

plt.xlim([0,nks-1]) # k-points
plt.ylim([ymin,ymax])
#plt.xlabel(r'$k (\AA^{-1})$',fontsize=16)
plt.ylabel('E-E$\mathrm{_F}$ (eV)',fontsize=16)

nvband = 17
eig_vbm = -(e_ref-0.6769)
eig_cbm = 2.3459-e_ref

for i in range(nbnd):
    line1=plt.plot( eig[:,i]-e_ref,color='black' ,linewidth=1)

#vlines= np.arange(0, nks, 20) # positions of vertical lines, or specified by [0, 20, 40, ...]
#for vline in vlines:
#    plt.axvline(x=vline, ymin=ymin, ymax=ymax,linewidth=lw,color='black')

plt.axvline(x=0, label = r'${\Gamma}$', color = 'black')
plt.axvline(x=30, label = 'M', color = 'black')
plt.axvline(x=50, label = 'K', color = 'black', zorder = 0)
plt.axvline(x=80, label = r'${\Gamma}$', color = 'black')

plt.arrow(50, 0, 0, eig_cbm, width = 0.2, color = 'red', length_includes_head=True, head_width=0.5, head_length=0.2, zorder = 1)
plt.arrow(50, 0, 0, eig_vbm, width = 0.2, color ='red', length_includes_head=True, head_width=0.5, head_length=0.2, zorder = 1)

#locs, labels = xticks()  # Get the current locations and labels.
plt.xticks([0, 30, 50, 80], [r'${\Gamma}$', 'M', 'K', r'${\Gamma}$'])

plt.axhline(y=0, label = 'Fermi Energy', ls='--', color ='blue')

fname = "bands.png"
band_plot.savefig(fname, dpi=500, transparent=True, bbox_inches='tight')